# Piano to Sheet

Converting .wav piano pieces into music sheets.

I

## Pipeline break down ( beta)

1. Loading in the wave file,
2. 



## Audio Processing  Libraries

In [ ]:
pip install tqdm

For the processing libraries, we will be using pretty_midi to extract out label information from the given midi files.  
And for processing our wav audio files. We will be using librosa.

## Machine Learning Lib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import librosa 
import json
import csv
import pandas as pd
import pretty_midi as pm
import mido
import IPython.display as ipd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import math
import time

# Model Saving funtion and Model loading function

Having a model saving function and a modle loading function in which we can train the model in small epoch progress.

# Preprocessing the data

In [ ]:

labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
with open(labels_file_path, 'r') as file:
    data = json.load(file)

all_sets = {}
all_sets['train'] = []
all_sets['validation'] = []
all_sets['test'] = []

def sortingsets (data, allsets):
    for key in data:
       
        if data[key] == 'train':
            all_sets['train'].append(key)
        elif data[key] == 'validation':
            all_sets['validation'].append(key)
        else:
            all_sets['test'].append(key)

def save_index_to_csv(all_sets):
    for key in all_sets:
        path = f"traindata/maestro-v3.0.0-midi/maestro-v3.0.0/{key}_indicies.csv"
        df = pd.DataFrame({'Index': all_sets[key]})
        df.to_csv(path, index=False)

def load_index_from_csv(path):
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices

def save_progress_index_to_csv(indices, epoch):
    path = f"models/training_index{epoch}.csv"
    df = pd.DataFrame({'Index': indices})
    df.to_csv(path, index=False)

def load_progress_index_from_csv(epoch):
    path = f"models/training_index{epoch}.csv"
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices

sortingsets(data['split'], all_sets)
save_index_to_csv(all_sets)




traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv

### Data Preprocessing
implenting utility functions such as the randomizing the data set for training.

In [ ]:
### making the random seed 
np.random.seed(40)

In [ ]:
## Randomizing the data set index for training purposes
def randomizeing(data_set):
    ds = np.array(data_set)
    np.random.shuffle(ds)
    return ds

## Select n indices from the givien data set
def select_N_randomized_from_set(n, data_set):
    nparry = randomizeing(data_set)
    return nparry[:n]


## ----- ----- ---------- loading function ------------------------------ ##

## function loading in the wav function
def load_wav_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    
    return librosa.load(wav_path, sr=None)

## function loading in the midi function
def load_midi_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return pm.PrettyMIDI(midi_path)

## ----- ----- -------- Path showing function-------------------------------- ##
    
## showing the file path audio of the wave
def show_wav_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    return wav_path

## Showing the file path of the midi file of data[index]
def show_midi_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return midi_path

## loading in the wav and midi pair
def load_wav_midi_pair(index): ## (wav, midi)
    return load_wav_from_index(index), load_midi_from_index(index)

In [ ]:
train_data = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
print(len(train_data))
subset_train_data = select_N_randomized_from_set(50, train_data)

print(len(subset_train_data))

# AUDIO EXPERIMENTATION

In [ ]:
(wav, sr), midi = load_wav_midi_pair(1025)
# midi = load_midi_from_index(1025)


In [ ]:
ballade1, sr = load_wav_from_index(505)

noised_ballade1 = add_gaussian_noise(ballade1, noise_level=0.01)

In [ ]:
print(len(ballade1))

In [ ]:
print(len(noised_ballade1))

In [ ]:
wav, sr = load_wav_from_index(216)


ipd.Audio(data= wav, rate= sr)


### Audio Preprocessing functions

#### Pretty_midi note
The MIDI object is used from the python package pretty_midi.  
Using pretty_midi range from 0 to 127. we can later tranform this into the range of 0 to 87 to match a piano

In [ ]:



""" This function extracts all played notes in the midi Object, which it will be futher trained with the aligne ed wave object
    input: pm object
    It is good for debugging and seeing the midi object
"""
def extract_midi_notes(midi):
    notes = []
    for instrument in midi.instruments:
        for note in instrument.notes:
            notes.append({
                'pitch': note.pitch,
                'start': note.start,
                'end': note.end,
                'velocity': note.velocity
            })
    #preprocessing the sort
    notes.sort(key=(lambda x: x['start']))
    return notes




""" Wrting a peekable Generator for midi object"""
class PeekableGenerator:
    def __init__(self, generator):
        self._generator = generator
        self._next_item = None
        self._has_next = False
        self._advance()

    def _advance(self):
        try:
            self._next_item = self._generator.__next__()
            self._has_next = True
        except StopIteration:
            self._next_item = None
            self._has_next = False

    def peek(self):
        if not self._has_next:
            raise StopIteration("No more elements to peek at.")
        return self._next_item

    def __next__(self):
        if not self._has_next:
            raise StopIteration("No more elements.")
        current = self._next_item
        self._advance()
        return current

    def has_next(self):
        return self._has_next

    def __iter__(self):
        yield self._next_item
        self._advance()

    

"""Generator to yield midi note object at the frame during the classification
    input: pm object
"""
def midi_yielding(midi):
    all_midi_obj :list = extract_midi_notes(midi)
    ##Processing
    for note in all_midi_obj:
        yield note


""" Yielding a list of midi notes information where it fits the time frame automatically.
    Implemented using overlapping frame structure for the training.
    Begin at 0, the frame jumping at the speed of jump_len, the size of the frame is frame_len
    This function will yield the frame at the given parameter.
"""
def frame_aligning_midi(t: int, jump_len:int, frame_len: int, sr:int, midi):
    midi_generator = PeekableGenerator(midi_yielding(midi))
    midi_labels = []

    jump_time_fraction: float = jump_len * (1/sr)
    frame_time_fraction: float = frame_len * (1/sr)
    framing = [t*jump_time_fraction, t*jump_time_fraction + frame_time_fraction]
    
    last_note = 0
    while midi_generator.has_next() or last_note > framing[0]:
        
        while midi_generator.has_next() and midi_generator.peek()['start'] >= framing[0] and midi_generator.peek()['start'] < framing[1]:
            try:
                midi_labels.append(midi_generator.__next__())
            except StopIteration:
                print("Generator exhausted, no Midi Objectis being added to the label")
                break
        
        ## Yielding the list of midi notes that are fitted in side the frame
        yield midi_labels

        midi_labels.sort(key=(lambda x: x['end']))
        ##calculating the next frame time step and removing the items from the previous frame
        framing [0] += jump_time_fraction
        framing [1] += jump_time_fraction
        while len(midi_labels) > 0 and midi_labels[0]['end'] < framing[0]:
            midi_labels.pop(0)
        
        if len(midi_labels) > 0 and midi_labels[0]['end'] > framing[0]:
            last_note = midi_labels[0]['end']

    
        
  



    
""" This function returns the aligned frame at the wav data,"""
def frame_aligning_wav(t: int, jump_len: int, frame_len: int, wav):
    begin = t * jump_len
    return wav[begin: begin + frame_len]


""" This function returns the amount seconds of audio data from the wav, began on t, while using  """

def audio_segment_of(t: int, wav, seconds: float, sr: int, jump_len: int = 512, frame_len: int = 2048, ):
    size = int(seconds*sr)
    begin = t*jump_len
    
    return wav[begin: begin + size]

def audio_segment_between(begin, end, wav, sr):
    return wav[int(begin*sr): int(end*sr)]

    


### Constructing Mel Spectrogram From wav frames

In [ ]:
def extract_mel_spectrogram(audio, sr, n_mels=88, hop_length=512, n_fft=4096): ##_fft is the frame length 
    """
    Extract a mel-spectrogram from raw audio.
    """
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft
    )
    # Convert to log scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db





In [ ]:
# Example audio input
frame_time = 2048 /sr
num_frame = 22
audio1 = audio_segment_between(0,2,ballade1, sr)
quick_sample = audio_segment_between(0, num_frame*frame_time, ballade1, sr)
mel_spectrogram = extract_mel_spectrogram(audio1, sr, n_mels =188)
print(mel_spectrogram.shape)


plt.figure(figsize=(25, 10))
librosa.display.specshow(mel_spectrogram, 
                         sr=sr, 
                         x_axis="linear")
plt.colorbar(format="%+2.f")
plt.show()
ipd.Audio(data= audio1, rate= sr)



### Calculating the added noise level to the wav function.

In [ ]:
def compute_noise_power(audio, snr_dB):
    """
    Compute the noise power needed for a given SNR in dB.
    :param audio: NumPy array of the audio signal.
    :param snr_dB: Desired Signal-to-Noise Ratio in dB.
    :return: Noise power.
    """
    # Calculate signal power (mean squared amplitude)
    signal_power = np.mean(audio ** 2)
    
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_dB / 10)
    
    # Calculate noise power
    noise_power = signal_power / snr_linear
    return noise_power





In [ ]:
def add_gaussian_noise(audio, noise_level=0.0006):
    
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise

def framelining (time, jump, sr):
    
    frametime = jump/sr
    print(22*frametime)



### Model for musheet


In [ ]:
class PianoNoteModel(nn.Module):
    def __init__(self, num_mel_bins=88, mel_temporal_length=89, num_frame_output=22, output_size=(88, 3)):
        """The default parameter is approximated for 1 seconds of audio data, regarding to the temporal_length"""
        super(PianoNoteModel, self).__init__()

        self.num_mel_bins = num_mel_bins
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.25)
        
        # Compute flattened size based on input dimensions after pooling
        # Assuming input shape is (batch_size, 1, num_mel_bins, num_frames)
        pooled_mel_bins = num_mel_bins // 2  # Adjust based on pooling
        pooled_temporal_length = mel_temporal_length // 2     # Adjust based on pooling

        
        flattened_size = pooled_mel_bins * pooled_temporal_length * 64  # Based on conv2 output channels
        
        
        # Fully connected layers
        self.fc1 = nn.Linear(flattened_size, 512)
        self.fc2 = nn.Linear(512, num_frame_output * output_size[0] * output_size[1])  # Predict for each frame
        
    def forward(self, x):
        # Input shape: (batch_size, 1, num_mel_bins, num_frames)
        
        # Convolutional layers
        
        x = F.relu(self.conv1(x))

       

        x = self.pool(F.relu(self.conv2(x)))
        
        x = self.dropout(x)
        
        # Flatten for fully connected layers
        x = x.view(x.size(0), -1)  # Flatten except batch dimension
       
        # Fully connected layers
        x = F.relu(self.fc1(x))
       
        x = self.fc2(x)
        
        # Reshape to output dimensions: (batch_size, num_frames, 88, 3)
        x = x.view(x.size(0), -1, 88, 3)
        
        return x
    

# Custom loss function, for mutipurpose loss function in the output layer
class MultiTaskLoss(nn.Module):
    def __init__(self, classification_weight=1.0, regression_weight=1.0):
        super(MultiTaskLoss, self).__init__()
        self.classification_loss = nn.BCEWithLogitsLoss(reduction='none')
        self.regression_loss = nn.MSELoss()
        self.classification_weight = classification_weight
        self.regression_weight = regression_weight

    def forward(self, classification_output, classification_target, 
                regression_output, regression_target, ratio):
        # Compute classification loss
        loss_per_element  = self.classification_loss(classification_output, classification_target)

        pressed_key_mask = classification_target == 1
        unpressed_key_mask = classification_target == 0

        pressed_key_label = classification_target[pressed_key_mask]
        unpressed_key_label = classification_target[unpressed_key_mask]


        pressed_keynumber = pressed_key_label.numel()
        unpressed_keynumber = unpressed_key_label.numel()


        # Compute the scoring of pressedkey and unpressed key with the feedback
        w_pressed = ((unpressed_keynumber/pressed_keynumber)**(ratio/2)) #* feed_back[0]
        w_unpressed= ((pressed_keynumber/unpressed_keynumber)**(ratio/2)) #* feed_back[1]
        #total =  pressed_keynumber + unpressed_keynumber
        #w_pressed = unpressed_keynumber /total
        #w_unpressed = pressed_keynumber / total

        weight_matrix = torch.full_like(classification_target, w_unpressed)
        weight_matrix[pressed_key_mask] = w_pressed
        
        weighted_class_loss_mat = loss_per_element * weight_matrix
        
        weighted_class_loss = weighted_class_loss_mat.sum()
        
        
        """# Computing the current accuracy of pressed and non pressed keys
        predicted_class = (classification_output > 0.5).float()
        correct_pressed = (predicted_class[pressed_key_mask] == classification_target[pressed_key_mask]).sum().item()
        correct_unpressed = (predicted_class[unpressed_key_mask] == classification_target[unpressed_key_mask]).sum().item()
        total_pressed = pressed_key_mask.sum().item()
        total_unpressed = unpressed_key_mask.sum().item()



        # update the feedback based on the accuracy
        epsilon = 1e-6  # Small constant to avoid instability
        correct_pressed_ratio = max(correct_pressed / total_pressed, epsilon)
        correct_unpressed_ratio = max(correct_unpressed / total_unpressed, epsilon)

        feed_back[0] = 1 / correct_pressed_ratio
        feed_back[1] = 1 / correct_unpressed_ratio"""

        # Compute regression loss
        reg_loss = self.regression_loss(regression_output, regression_target)
        
        # Combine with weights
        total_loss = self.classification_weight * weighted_class_loss + self.regression_weight * reg_loss
        
        return total_loss
    
    def set_classification_balancer(self, weight):
        self.classification_loss.weight = weight
    




### Training functions

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import tqdm

## MODEL Saver
def save_model(model, optimizer, id, path = "models/"):
    """ Saving the model after each training/testing before each training progress"""
    actualPath = path + f"{id}_piano_model.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, actualPath)
    print(f"Model saved to {actualPath}")

## MODEL loader
def load_model(model, optimizer, id, path = "models/"):
    actualPath = path + f"{id}_piano_model.pth"
    checkpoint = torch.load(actualPath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer


"""This function generates the matrix label to aligne the mal_spectral gram
each matrix should consired num_frame,
poping fram_jumping after the matrix is being yield

jumping time should be
frame_jumping * (frame_len /sr)
"""
def label_generator(num_frame, frame_jumping, jump_len:int, frame_len: int, sr:int, midi):
    midi_frame_gen = PeekableGenerator(frame_aligning_midi(0, frame_len, frame_len, sr, midi)) ## Note, while using the architechture of mel_spectrogram, the we don't need to consider the jump offset in the midi side
    label = []
    counter = 0
    
    
    concur_time = 0
    concur_time_end = 0
    frame_time = frame_len /sr

    
    while midi_frame_gen.has_next():
        if (len(label) < num_frame):
            label.append(list(midi_frame_gen.__next__()))
            concur_time_end += frame_time
            
        else:
            counter+=1
            yield label
            for i in range(int(frame_jumping)):
                label.pop(0)
                concur_time += frame_time
                


"""This function tries to mimic the decayed velocity miniking the sound at which a piano has been decayed"""
def velocity_decay_sustain (velocity, onset, at_time):
    if (at_time - onset < 0.2):
        return velocity
    else:
        return math.exp((onset - at_time) * 0.6) * velocity


"""formating the label of list, in to a matrix of 88 * 3 matrix.
    each row represent a strikable key, 
    column 1 (being stricked) : 0, 1 (classification purpose)
    column 2 (onset timer) : the set of positive interger that is less than onset. (Regression purpose)
    column 3 (velocity of which is being stricked) : the set of positive integer that is less than onset. (Regression purpose) 
        note for the velocity of the piano key will be approximately alingned with a decay parameter"""
def label_formater(label, frameonset):
    ret_label = np.zeros((len(label), 88,3))

    for i in range(len(label)) :
        for key_obj in label[i]:
            pitch = key_obj['pitch'] - 20
            ret_label[i][pitch][0] = 1
            ret_label[i][pitch][1] = key_obj['start'] - frameonset if key_obj['start'] > frameonset else 0
            ret_label[i][pitch][2] = velocity_decay_sustain(key_obj['velocity'],  key_obj['start'], frameonset) 

    return ret_label




"""This function assure the input for the training will retain the dimension in the case of track is ending."""
def construct_input(spectrogram, x, y):
    if spectrogram.shape == (x,y):
        return spectrogram
    else: 
        ret = np.zeros((x,y))
        
       
        for i in range(spectrogram.shape[0]):
            ret[i][:spectrogram.shape[1]] = spectrogram[i][:spectrogram.shape[1]]
        return ret

def train_segment(wav, sr, length=1, hop=0.5):
    ret = []
    max = len(wav)/sr
    begin = 0
    while (begin < max):
        adding = length if begin + length < max else max - begin
        ret.append((begin, begin + adding))
        begin += hop

    return ret
    

def label_buffer(num_frame, label):
    if len(label) < num_frame:
        for i in range(num_frame- len(label)):
            label.append([])
    return label


""" This function will generate all the mel_spectrogram and label pair for each audio segment of the song at index"""
def generate_data_label(index, noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512):
    # fetching infomation
    (wav, sr), midi = load_wav_midi_pair(index)
    
    if noised:
        wav = add_gaussian_noise(wav)

    frame_time =  frame_length/sr
    quick_sample = audio_segment_between(0, num_frame*frame_time, wav, sr)
    x, y = extract_mel_spectrogram(quick_sample, sr).shape
    segments = train_segment(wav, sr, length = num_frame * frame_time, hop= num_frame * frame_time/2)
    
    label_gen = PeekableGenerator(label_generator(num_frame, num_frame*segment_jump, hop_length, frame_length, sr, midi))
    
    input_datas = []
    labels = []
    for beg, end in segments:
        
        if (label_gen.has_next() is False) :
            break

        audio = audio_segment_between(beg, end, wav, sr)
        mel_spectrogram = extract_mel_spectrogram(audio, sr, hop_length= hop_length, n_fft=frame_length)
        input_data = construct_input(mel_spectrogram, x,y)
        input_data = np.array(input_data)
        input_data = np.expand_dims(input_data, axis=0)

        label = label_buffer(num_frame, label_gen.__next__())
        label = label_formater(label, beg)
        label = np.array(label)

        input_datas.append(input_data)
        labels.append(label)
        
    return np.array(input_datas), np.array(labels)
   
def generate_data(index, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512):
    """This function generates the audio mel_spectrogram for the song at {index} with the defaulted training parameter
        this function is generally used for validation process.
    """
    # fetching infomation
    (wav, sr), midi = load_wav_midi_pair(index)
    wav, sr = load_wav_from_index(index)
    
    frame_time =  frame_length/sr
    quick_sample = audio_segment_between(0, num_frame*frame_time, wav, sr)
    x, y = extract_mel_spectrogram(quick_sample, sr).shape
    segments = train_segment(wav, sr, length = num_frame * frame_time, hop= num_frame * frame_time/2)
    input_datas = []
 
    for beg, end in segments:
        audio = audio_segment_between(beg, end, wav, sr)
        mel_spectrogram = extract_mel_spectrogram(audio, sr, hop_length= hop_length, n_fft=frame_length)
        input_data = construct_input(mel_spectrogram, x,y)
        input_data = np.array(input_data)
        input_data = np.expand_dims(input_data, axis=0)   
        input_datas.append(input_data)
        
    return np.array(input_datas)



def one_pass_song_train(id, index, ratio,  noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048
                        , hop_length = 512, batch_size=16 ):
    #load model
    """The training is based on how many frame should be trained at a time,
        the default setting is suited for the expriment set up above,
        num_frame is tried to aligned it to approx 1 second of the sample
        segment_jumping would be trying to get 50% of the sample audio
        The dimension of the input is:
        batchsize * channel * 
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    #Letting the classification utility be more potent than the regression utility.
    criterion = MultiTaskLoss(classification_weight=1.7, regression_weight=0.5)
 
    
    model, optimizer = load_model(model, optimizer, id)

    # generate all data
    datas, labels = generate_data_label(index, noised=noised, num_frame=num_frame, segment_jump=segment_jump, frame_length=frame_length, hop_length=hop_length)
    datas = torch.tensor(datas, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(datas, labels)
    # Let model be in training mode
    model.train()
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle =True)
    progress_bar = tqdm.tqdm(enumerate(dataloader), total=len(dataloader))
    # Initialize the feedback value before training to 
    

    lossval = 0.0
    for _, (batch_inputs, batch_labels) in progress_bar:
    
        optimizer.zero_grad()

        outputs = model(batch_inputs)

        #  classification training
        classification_output = outputs[..., 0:1]
        classification_target = batch_labels[..., 0:1]
        
        #  regression training
        regression_output = outputs[..., 1:3]
        regression_target = batch_labels[...,1:3]
        loss = criterion.forward(classification_output, classification_target, regression_output, regression_target, ratio)
    
        

        loss.backward()
        optimizer.step()

        lossval+= loss.item()

        
    #save model
    save_model(model, optimizer, id)
    return lossval





def train_N_song_on_epoch (id, epoch, N, repetition = 2, noised=True):
    

    tobe_done = load_progress_index_from_csv(epoch)

    ## indices to be trained for the current run
    parse_in = [tobe_done.pop() for i in range(N)]

    feed_back_tensor = feed_back = torch.tensor([1.0,1.0])
    
    ## Passing in each index to train on one_pass_song_train
    for index in parse_in:
        one_pass_song_train(id, index)

    print("Message from train_N_song_on_epoch") 
    if (len(tobe_done) == 0):
        print(f"Epoch {epoch} training complete")
    else :
        
        print(f"Epoch {epoch} have trained {N} songs, left over songs for current epoch will be saved. Need to train {len(tobe_done)}.")
        save_progress_index_to_csv(tobe_done, epoch)





### Validation functions

Use the function below to test the accuracy of the model

In [83]:

def validation_accuracy_check(id, index, noised=True, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512, batch_size=16):
    """This function returns the accuracy of the model predicting the song at index
        arguments:
            id: trained model id
            index: index of the song.
    """
    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    #Letting the classification utility be more potent than the regression utility.
    criterion = MultiTaskLoss(classification_weight=1.5, regression_weight=0.7)
 
    
    model, optimizer = load_model(model, optimizer, id)
   

    datas, labels = generate_data_label(index, num_frame=num_frame, segment_jump=segment_jump, frame_length=frame_length, hop_length=hop_length)
    

    datas = torch.tensor(datas, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(datas, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    classification_correct = 0
    classification_true_correct = 0
    regression_error = 0

    true_classified = 0
    total_classifed = 0
    num_classified = 0
    
    with torch.no_grad():
        
        for val_inputs, val_labels in dataloader:
            outputs = model(val_inputs)

            #Output separation
            classification_outputs = outputs[:,:,:, 0:1].squeeze(-1)
            regression_outputs = outputs[:, :, :, 1:3]

            #Label separation
            classification_labels = val_labels[:, :, :, 0:1].squeeze(-1)
            regression_labels = val_labels[:, :, :, 1:3]
            
            #Calculate the classification accuracy of a frame
            predicted_classes = (classification_outputs > 0.5).float()
            classification_correct += (predicted_classes == classification_labels).sum().item() / model.num_mel_bins #this should be the accuracy of a frame


            #Calculate the classification accuracy of pressed key
            pressed_key_mask = classification_labels == 1
            pressed_key_label = classification_labels[pressed_key_mask]
            pressed_key_prediction = predicted_classes[pressed_key_mask]
            true_classified += pressed_key_label.numel()
            total_classifed += classification_labels.numel()
            classification_true_correct += (pressed_key_label == pressed_key_prediction).sum().item()
            

            regression_error += ((regression_outputs - regression_labels) ** 2).mean().item()
            
            num_classified += val_labels.shape[0] * val_labels.shape[1] # counting number of frames have been classified i.e. batch_number * number of frame per segment
            

    if true_classified > 0 :
        true_accuracy = classification_true_correct/true_classified
    else:
        true_accuracy = 0
    print(f"Classified {num_classified} frames of audio, classification arcuracy {classification_correct/ num_classified}, average regression error is {regression_error/ num_classified}")
    print(f"The true accuracy of classifying pressed_key are {true_accuracy}")
    print(f"Ratio of pressed is in data is {true_classified/total_classifed}")
    return num_classified, classification_correct, regression_error


    

def model_output(id, index):
    """ This function returns the audio classification from the model {id}
        arguments : id (model id)
                    index (song index)
            return: classification result
    """
    model = PianoNoteModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    model, _ = load_model(model, optimizer, id)
    audio_data = generate_data(index)


    datas = torch.tensor(audio_data, dtype=torch.float32)


    dataloader = DataLoader(audio_data, batch_size = 16, shuffle =True)
    


    dataset = TensorDataset(datas, labels)
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle =True)

    
    progress_bar = tqdm.tqdm(enumerate(dataloader), total=len(dataloader))

    epoch_loss = 0.0
    for batch_inputs, batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)




def midi_reconstruction(outputed_data):
    """Reconstruct the midi object from the outputed_data.
        Arguments:
            outputed_data: A list of outputed matrixies[batch_size * frame_number * 88 * 3]
    """





# Training

### Index Processing

Using the saved indices we have processed before  
Here are the file paths  
for training datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
for test datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
for validataion datas :traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv  

In [ ]:
## Only run this once
train_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
validation_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv")
test_indices = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv")

def initiate_train_epoch(epoch_Number, dataset):
    train_indices = load_index_from_csv(dataset)
    train_indices = randomizeing(train_indices)
    save_progress_index_to_csv(train_indices, epoch_Number)


In [ ]:
# Initiate the training epoch. RUN ONCE BEFORE TRAINING
#initiate_train_epoch(0, "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")

# Initiate the training model. RUN ONCE BEFORE TRAINING
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "test")

## Trainign prototype
Experimenting training on one single sample to see if there are any progresses.

In [ ]:
indicies = [686, 1272, 216, 1024, 962, 505]

In [ ]:
for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")
    
a, b, c = validation_accuracy_check("prototype", 2)

In [ ]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype3")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype3", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype3", 2)

In [ ]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype0")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype0", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype0", 2)

In [ ]:
a, b, c = validation_accuracy_check("prototype0", 9 )

In [63]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype01")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype01", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype01", 2)

Model saved to models/prototype01_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\2953729452.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype01_piano_model.pth
56083264517.15625


100%|██████████| 15/15 [00:06<00:00,  2.27it/s]


Model saved to models/prototype01_piano_model.pth
3209031.17578125


...

Model saved to models/prototype01_piano_model.pth
1933284633600.0


100%|██████████| 67/67 [00:27<00:00,  2.44it/s]


Model saved to models/prototype01_piano_model.pth
706383815805.5


100%|██████████| 67/67 [00:27<00:00,  2.46it/s]


Model saved to models/prototype01_piano_model.pth
140088208456.75
final loss val for 505 is 140088208456.75
Classified 21758 frames of audio, classification arcuracy 0.9654181534064799, average regression error is 0.19379167871688238
The true accuracy of classifying pressed_key are 0.0033094148212297414


In [65]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback", 2)

Model saved to models/prototype_nofeedback_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\2953729452.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_piano_model.pth
6799158.517578125


100%|██████████| 15/15 [00:06<00:00,  2.44it/s]


Model saved to models/prototype_nofeedback_piano_model.pth
390816.9296875


...

Model saved to models/prototype_nofeedback_piano_model.pth
406124.0244140625


100%|██████████| 67/67 [00:26<00:00,  2.49it/s]


Model saved to models/prototype_nofeedback_piano_model.pth
321300.5856933594


100%|██████████| 67/67 [00:26<00:00,  2.53it/s]


Model saved to models/prototype_nofeedback_piano_model.pth
311983.3181152344
final loss val for 505 is 311983.3181152344
Classified 21758 frames of audio, classification arcuracy 0.23040428181066114, average regression error is 0.18628961889388115
The true accuracy of classifying pressed_key are 0.9920821477174316


In [74]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_batchrep")

for index in indicies:
    final_loss = 0
    for _ in range(3):
        train_lost_val = one_pass_song_train("prototype_nofeedback_batchrep", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_batchrep", 2)

Model saved to models/prototype_nofeedback_batchrep_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\1372277917.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_batchrep_piano_model.pth
1970549.6208496094


100%|██████████| 15/15 [00:12<00:00,  1.20it/s]


Model saved to models/prototype_nofeedback_batchrep_piano_model.pth
149679.1251220703


...

Model saved to models/prototype_nofeedback_batchrep_piano_model.pth
372390.2529296875


100%|██████████| 67/67 [00:53<00:00,  1.24it/s]


Model saved to models/prototype_nofeedback_batchrep_piano_model.pth
325973.0985107422


100%|██████████| 67/67 [00:52<00:00,  1.27it/s]


Model saved to models/prototype_nofeedback_batchrep_piano_model.pth
323627.29553222656
final loss val for 505 is 323627.29553222656
Classified 21758 frames of audio, classification arcuracy 0.20748585682173334, average regression error is 0.1865961725017759
The true accuracy of classifying pressed_key are 0.9963967586292218


In [86]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback1")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback1", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback1", 2)

Model saved to models/prototype_nofeedback1_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback1_piano_model.pth
2355270.1752929688


100%|██████████| 15/15 [00:06<00:00,  2.37it/s]


Model saved to models/prototype_nofeedback1_piano_model.pth
193579.07739257812


...

Model saved to models/prototype_nofeedback1_piano_model.pth
294349.59765625


100%|██████████| 67/67 [00:26<00:00,  2.50it/s]


Model saved to models/prototype_nofeedback1_piano_model.pth
245903.02111816406


100%|██████████| 67/67 [00:26<00:00,  2.51it/s]


Model saved to models/prototype_nofeedback1_piano_model.pth
229524.08837890625
final loss val for 505 is 229524.08837890625
Classified 21758 frames of audio, classification arcuracy 0.3527537415704986, average regression error is 0.18545492647193637
The true accuracy of classifying pressed_key are 0.9719472967957442
Ratio of pressed is in data is 0.033772321988150646


In [84]:
a, b, c = validation_accuracy_check("prototype_nofeedback1", 2)

C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)


Classified 21758 frames of audio, classification arcuracy 0.20990085151543, average regression error is 0.18621120444991285
The true accuracy of classifying pressed_key are 0.9962575776320673
Ratio of pressed is in data is 0.033772321988150646


In [88]:
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1:0.75")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1:0.75", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_inver1:0.75", 2)

Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth
1832804.3347167969


100%|██████████| 15/15 [00:06<00:00,  2.32it/s]


Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth
183632.38256835938


...

Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth
305548.1257324219


100%|██████████| 67/67 [00:27<00:00,  2.48it/s]


Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth
260913.677734375


100%|██████████| 67/67 [00:27<00:00,  2.46it/s]


Model saved to models/prototype_nofeedback_inver1:0.75_piano_model.pth
255317.9970703125
final loss val for 505 is 255317.9970703125
Classified 21758 frames of audio, classification arcuracy 0.2699252730448153, average regression error is 0.18600080707776917
The true accuracy of classifying pressed_key are 0.9883087962390201
Ratio of pressed is in data is 0.033772321988150646


In [ ]:
"""model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1:0.6")"""

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1:0.6", index, 1.6)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_inver1:0.6", 2)

Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth
970016.8449707031


100%|██████████| 15/15 [00:06<00:00,  2.29it/s]


Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth
134986.41967773438


...

Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth
244876.7774658203


100%|██████████| 67/67 [00:28<00:00,  2.38it/s]


Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth
220074.19274902344


100%|██████████| 67/67 [00:26<00:00,  2.48it/s]


Model saved to models/prototype_nofeedback_inver1:0.6_piano_model.pth
211189.76904296875
final loss val for 505 is 211189.76904296875
Classified 21758 frames of audio, classification arcuracy 0.3747529644268776, average regression error is 0.185820740366565
The true accuracy of classifying pressed_key are 0.9604416676976371
Ratio of pressed is in data is 0.033772321988150646


In [102]:
"""model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1:0.5")"""

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1:0.5", index)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")



C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1:0.5_piano_model.pth
50881.95782470703


100%|██████████| 15/15 [00:06<00:00,  2.32it/s]


Model saved to models/prototype_nofeedback_inver1:0.5_piano_model.pth
43366.88195800781


100%|██████████| 15/15 [00:06<00:00,  2.34it/s]


...

Model saved to models/prototype_nofeedback_inver1:0.5_piano_model.pth
170859.40368652344


100%|██████████| 67/67 [00:26<00:00,  2.48it/s]


Model saved to models/prototype_nofeedback_inver1:0.5_piano_model.pth
149027.56518554688


100%|██████████| 67/67 [00:27<00:00,  2.44it/s]


Model saved to models/prototype_nofeedback_inver1:0.5_piano_model.pth
139293.43920898438
final loss val for 505 is 139293.43920898438


In [95]:
a, b, c = validation_accuracy_check("prototype_nofeedback_inver1:0.5", 2)

C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)


Classified 21758 frames of audio, classification arcuracy 0.4214238858852335, average regression error is 0.18541228469871776
The true accuracy of classifying pressed_key are 0.9379716689348014
Ratio of pressed is in data is 0.033772321988150646


In [101]:
# the ratio is np/p
"""model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_inverse_fraction")"""

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_inverse_fraction", index, 1.0)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_inverse_fraction", 2)

C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_inverse_fraction_piano_model.pth
71897.5269165039


100%|██████████| 15/15 [00:06<00:00,  2.40it/s]


Model saved to models/prototype_inverse_fraction_piano_model.pth
60413.78088378906


100%|██████████| 15/15 [00:06<00:00,  2.41it/s]


...

Model saved to models/prototype_inverse_fraction_piano_model.pth
186144.37817382812


100%|██████████| 67/67 [00:27<00:00,  2.42it/s]


Model saved to models/prototype_inverse_fraction_piano_model.pth
150751.97912597656


100%|██████████| 67/67 [00:26<00:00,  2.49it/s]


Model saved to models/prototype_inverse_fraction_piano_model.pth
139014.96520996094
final loss val for 505 is 139014.96520996094
Classified 21758 frames of audio, classification arcuracy 0.5504981448829687, average regression error is 0.18436247032018957
The true accuracy of classifying pressed_key are 0.8865520227638253
Ratio of pressed is in data is 0.033772321988150646


In [104]:
a, b, c = validation_accuracy_check("prototype_nofeedback_inver1:0.5", 2)


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)


Classified 21758 frames of audio, classification arcuracy 0.5487829972674627, average regression error is 0.18461263969832345
The true accuracy of classifying pressed_key are 0.8737628355808487
Ratio of pressed is in data is 0.033772321988150646


In [100]:
# the ratio is (np/p)**1.4
model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1_04")

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1_04", index, 1.4)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_inver1_04", 2)

Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth


C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth
442000.3289794922


100%|██████████| 15/15 [00:06<00:00,  2.36it/s]


Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth
69669.31188964844


...

Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth
194215.29833984375


100%|██████████| 67/67 [00:26<00:00,  2.52it/s]


Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth
174008.4061279297


100%|██████████| 67/67 [00:27<00:00,  2.46it/s]


Model saved to models/prototype_nofeedback_inver1_04_piano_model.pth
166102.2537841797
final loss val for 505 is 166102.2537841797
Classified 21758 frames of audio, classification arcuracy 0.46278589275418036, average regression error is 0.18587493631306987
The true accuracy of classifying pressed_key are 0.9010577755783744
Ratio of pressed is in data is 0.033772321988150646


In [107]:
# the ratio is (np/p)**1.4
"""model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1_03")"""

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1_03", index, 1.3)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_inver1_03", 2)

C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1_03_piano_model.pth
46699.016174316406


100%|██████████| 15/15 [00:06<00:00,  2.28it/s]


Model saved to models/prototype_nofeedback_inver1_03_piano_model.pth
39016.36114501953


100%|██████████| 15/15 [00:06<00:00,  2.41it/s]


...

Model saved to models/prototype_nofeedback_inver1_03_piano_model.pth
157921.3243408203


100%|██████████| 67/67 [00:27<00:00,  2.44it/s]


Model saved to models/prototype_nofeedback_inver1_03_piano_model.pth
131148.43908691406


100%|██████████| 67/67 [00:27<00:00,  2.40it/s]


Model saved to models/prototype_nofeedback_inver1_03_piano_model.pth
119024.27429199219
final loss val for 505 is 119024.27429199219
Classified 21758 frames of audio, classification arcuracy 0.5994869180823769, average regression error is 0.1857240335261277
The true accuracy of classifying pressed_key are 0.8078683657058023
Ratio of pressed is in data is 0.033772321988150646


In [106]:
"""model = PianoNoteModel(output_size=(88, 3))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
save_model(model, optimizer, "prototype_nofeedback_inver1_02")"""

for index in indicies:
    final_loss = 0
    for i in range(3): 
        train_lost_val = one_pass_song_train("prototype_nofeedback_inver1_02", index, 1.2)
        print(train_lost_val)
        final_loss = train_lost_val
    print(f"final loss val for {index} is {final_loss}")

a, b, c = validation_accuracy_check("prototype_nofeedback_inver1_02", 2)

C:\Users\youxu\AppData\Local\Temp\ipykernel_9368\3530722454.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(actualPath)
100%|██████████| 15/15 [

Model saved to models/prototype_nofeedback_inver1_02_piano_model.pth
54125.751525878906


100%|██████████| 15/15 [00:06<00:00,  2.28it/s]


Model saved to models/prototype_nofeedback_inver1_02_piano_model.pth
45380.26171875


100%|██████████| 15/15 [00:06<00:00,  2.35it/s]


...

Model saved to models/prototype_nofeedback_inver1_02_piano_model.pth
177592.96032714844


100%|██████████| 67/67 [00:27<00:00,  2.41it/s]


Model saved to models/prototype_nofeedback_inver1_02_piano_model.pth
165854.9178466797


100%|██████████| 67/67 [00:27<00:00,  2.45it/s]


Model saved to models/prototype_nofeedback_inver1_02_piano_model.pth
161723.16430664062
final loss val for 505 is 161723.16430664062
Classified 21758 frames of audio, classification arcuracy 0.47476424554395863, average regression error is 0.18519864102936984
The true accuracy of classifying pressed_key are 0.9035475689719163
Ratio of pressed is in data is 0.033772321988150646
